# 使用BDMLtools做策略分析报告 <font size=3 >by 曾珂</font>

In [1]:
import BDMLtools as bm
import pandas as pd
import numpy as np

In [2]:
bm.__version__

'0.1.7'

+ load germancredit data

In [3]:
import scorecardpy as sc
dt=sc.germancredit().copy()
dt['creditability']=dt['creditability'].map({'good':0,'bad':1})

+ 将germancredit的数据类型进行指定
    + num列-->float64
    + str列-->object
    + date列-->datetime

In [4]:
dtypes_dict={
    'num':['age.in.years',
         'credit.amount',
         'creditability',
         'duration.in.month',
         'installment.rate.in.percentage.of.disposable.income',
         'number.of.existing.credits.at.this.bank',
         'number.of.people.being.liable.to.provide.maintenance.for',
         'present.residence.since'],
    'str':['housing','telephone','foreign.worker','purpose','job','personal.status.and.sex','property',
           'credit.history','savings.account.and.bonds','present.employment.since',
           'status.of.existing.checking.account',
           'other.installment.plans','other.debtors.or.guarantors'],
    'date':[]
}

In [5]:
da=bm.dtypeAllocator(dtypes_dict=dtypes_dict).fit(dt)
dt=da.transform(dt)
X=dt.drop('creditability',axis=1)
y=dt['creditability']

## BDMLtools中的报告

实践进行策略分析时，有时希望快速生成策略特征的分析报告,BDMLtools提供了三种特征分析库:
    
+ varReport:特征分析报告
+ varReportSinge:单特征分析报告
+ varGroupsReport:组特征分析报告

## varReport

### 基本用法

一般特征分析需先进行分箱再生成报告,这里使用binSelector进行最优分箱

In [6]:
bin_tree=bm.binSelector(method='tree',bin_num_limit=8,n_jobs=1,iv_limit=0).fit(X,y)

+ bin_tree中的属性breaks_list为分箱字典,包含了所有特征的分箱结果,格式与scorecardpy一致
+ 将breaks_list作为varReport的入参以按照指定的breaks_list分箱产生特征分析报告

In [7]:
vtab=bm.varReport(bin_tree.breaks_list,n_jobs=1).fit(X,y)

+ vtab的var_report_dict保存了所有特征的分箱结果,结构与scorecardpy一致

In [8]:
vtab.var_report_dict.keys()

dict_keys(['age.in.years', 'credit.amount', 'credit.history', 'duration.in.month', 'foreign.worker', 'housing', 'installment.rate.in.percentage.of.disposable.income', 'job', 'number.of.existing.credits.at.this.bank', 'number.of.people.being.liable.to.provide.maintenance.for', 'other.debtors.or.guarantors', 'other.installment.plans', 'personal.status.and.sex', 'present.employment.since', 'present.residence.since', 'property', 'purpose', 'savings.account.and.bonds', 'status.of.existing.checking.account', 'telephone'])

In [9]:
vtab.var_report_dict['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,105,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,60,0.060,39.0,21.0,0.350000,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,242,0.242,182.0,60.0,0.247934,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,80,0.080,66.0,14.0,0.175000,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,58,0.058,31.0,27.0,0.465517,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,138,0.138,89.0,49.0,0.355072,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


这里每一个报表中bin索引,
+ 存在missing行用于标示缺失值,数据中的np.nan将被计算进缺失值行中,若数据没有缺失值则为默认值
+ special行用于标示除缺失值外的特殊编码,可通过special_values参数进行指定,若数据没有特殊值则为默认值,详情请见文档

In [10]:
pd.concat(vtab.var_report_dict)

variable  \
                                    bin                                                                             
age.in.years                        [-inf, 26.0)                                                     age.in.years   
                                    [26.0, 28.0)                                                     age.in.years   
                                    [28.0, 30.0)                                                     age.in.years   
                                    [30.0, 35.0)                                                     age.in.years   
                                    [35.0, 37.0)                                                     age.in.years   
...                                                                                                           ...   
status.of.existing.checking.account missing                                   status.of.existing.checking.account   
telephone                           yes, registered under the customers name                            telephone   
                                    none                                                                telephone   
                                    special                                                             telephone   
                                    missing                                                             telephone   

                                                                              count  \
                                    bin                                               
age.in.years                        [-inf, 26.0)                                190   
                                    [26.0, 28.0)                                101   
                                    [28.0, 30.0)                                 80   
                                    [30.0, 35.0)                                177   
                                    [35.0, 37.0)                                 79   
...                                                                             ...   
status.of.existing.checking.account missing                                       0   
telephone                           yes, registered under the customers name    404   
                                    none                                        596   
                                    special                                       0   
                                    missing                                       0   

                                                                              count_distr  \
                                    bin                                                     
age.in.years                        [-inf, 26.0)                                    0.190   
                                    [26.0, 28.0)                                    0.101   
                                    [28.0, 30.0)                                    0.080   
                                    [30.0, 35.0)                                    0.177   
                                    [35.0, 37.0)                                    0.079   
...                                                                                   ...   
status.of.existing.checking.account missing                                         0.000   
telephone                           yes, registered under the customers name        0.404   
                                    none                                            0.596   
                                    special                                         0.000   
                                    missing                                         0.000   

                                                                               good  \
                                    bin                                               
age.in.years                        [-inf, 26.0)                   

使用pd.concat形成所有特征的特征分析报告,与scorecardpy一致

### 样本权重

若数据是经过抽样获取的，一般希望通过样本加权建模以利于还原其违约概率

+ 加入样本权重后各个分箱的坏样本率会产生一定变化
+ 加入样本权重后各个分箱的iv,ks也会产生变化(若只对好坏样本加权且breaks一致则无变化,因为好坏分布未变化)

BDMLtools的报告支持加入样本权重

In [11]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index) #假定数据经过抽样后好样本权重为10,坏样本权重为1
sample_weight_oth=pd.Series(np.random.randint(0,100,y.size)/100,index=y.index) #模拟复杂抽样情形下的样本权重

In [12]:
vtab_ws=bm.varReport(bin_tree.breaks_list,n_jobs=1,sample_weight=sample_weight).fit(X,y)
vtab_ws_oth=bm.varReport(bin_tree.breaks_list,n_jobs=1,sample_weight=sample_weight_oth).fit(X,y)

In [13]:
vtab_ws.var_report_dict['credit.amount'] #加权后(仅类加权)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,1932,0.264658,1850.0,82.0,0.042443,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,888,0.121644,870.0,18.0,0.020270,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,411,0.056301,390.0,21.0,0.051095,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,1880,0.257534,1820.0,60.0,0.031915,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,674,0.092329,660.0,14.0,0.020772,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,337,0.046164,310.0,27.0,0.080119,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,939,0.128630,890.0,49.0,0.052183,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,239,0.032740,210.0,29.0,0.121339,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


In [14]:
vtab_ws_oth.var_report_dict['credit.amount'] #加权后(样本加权)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,130.86,0.261031,88.22,42.64,0.325844,0.119649,0.003824,0.296057,3.195826e-02,0.166559,1400.0
"[1400.0, 1800.0)",credit.amount,54.77,0.109252,46.43,8.34,0.152273,-0.870193,0.066920,0.296057,4.494371e-02,0.166559,1800.0
"[1800.0, 2000.0)",credit.amount,26.30,0.052462,19.08,7.22,0.274525,-0.125096,0.000800,0.296057,5.133820e-02,0.166559,2000.0
"[2000.0, 3400.0)",credit.amount,130.27,0.259854,97.90,32.37,0.248484,-0.260024,0.016613,0.296057,1.152266e-01,0.166559,3400.0
"[3400.0, 4000.0)",credit.amount,42.30,0.084377,35.01,7.29,0.172340,-0.722440,0.037084,0.296057,1.665586e-01,0.166559,4000.0
"[4000.0, 5000.0)",credit.amount,29.32,0.058486,14.82,14.50,0.494543,0.824861,0.044651,0.296057,1.124265e-01,0.166559,5000.0
"[5000.0, 9200.0)",credit.amount,67.86,0.135363,43.14,24.72,0.364279,0.289852,0.011983,0.296057,7.108542e-02,0.166559,9200.0
"[9200.0, inf)",credit.amount,19.64,0.039177,6.26,13.38,0.681263,1.606271,0.114182,0.296057,1.110223e-16,0.166559,inf
special,credit.amount,0.00,0.000000,0.00,0.00,NaN,0.000000,0.000000,0.296057,1.110223e-16,0.166559,special


In [15]:
vtab.var_report_dict['credit.amount'] #未加权

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,105,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,60,0.060,39.0,21.0,0.350000,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,242,0.242,182.0,60.0,0.247934,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,80,0.080,66.0,14.0,0.175000,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,58,0.058,31.0,27.0,0.465517,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,138,0.138,89.0,49.0,0.355072,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


### 并行

varReport使用joblib对列的报告生成进行了并行优化,
+ 当基础的数据量非常大列较多时可通过设定n_jobs=任务数进行并行以提高运行速度
+ 一般情况下建议n_jobs=1

In [47]:
#模拟较大大数据量,3w行1000个特征
X_big=pd.DataFrame(np.random.rand(30000,1000),columns=['f'+str(i) for i in range(1000)])
y_big=pd.Series(np.random.randint(0,2,30000),name='target')
breaks_list_big={col:[0.2,0.4,0.6,0.8] for col in X_big.columns}

In [48]:
X_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1000 entries, f0 to f999
dtypes: float64(1000)
memory usage: 228.9 MB


In [49]:
%%time
res_p=bm.varReport(breaks_list_big,n_jobs=-1).fit(X_big,y_big) 

CPU times: user 850 ms, sys: 160 ms, total: 1.01 s
Wall time: 5.92 s


In [50]:
%%time
res=bm.varReport(breaks_list_big,n_jobs=1).fit(X_big,y_big)

CPU times: user 14.1 s, sys: 104 ms, total: 14.2 s
Wall time: 14.3 s


In [51]:
#结果一致
pd.concat(res.var_report_dict).equals(pd.concat(res_p.var_report_dict))

True

## varReportSinge

### 基本用法

对单一特征产生报告可用bm.varReportSinge,其可以灵活得调整分箱

In [21]:
bm.varReportSinge().report(X['age.in.years'],y,[20,30,40,50])

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 20.0)",age.in.years,2,0.002,1.0,1.0,0.500000,0.847298,0.001614,0.064422,0.001905,0.122381,20.0
"[20.0, 30.0)",age.in.years,369,0.369,233.0,136.0,0.368564,0.308914,0.037217,0.064422,0.122381,0.122381,30.0
"[30.0, 40.0)",age.in.years,330,0.330,245.0,85.0,0.257576,-0.211309,0.014087,0.064422,0.055714,0.122381,40.0
"[40.0, 50.0)",age.in.years,174,0.174,130.0,44.0,0.252874,-0.236047,0.009217,0.064422,0.016667,0.122381,50.0
"[50.0, inf)",age.in.years,125,0.125,91.0,34.0,0.272000,-0.137201,0.002287,0.064422,0.000000,0.122381,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.064422,0.000000,0.122381,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.064422,0.000000,0.122381,missing


In [22]:
bm.varReportSinge().report(X['age.in.years'],y,[25,35,45,55])

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 25.0)",age.in.years,149,0.149,88.0,61.0,0.409396,0.480835,0.037322,0.089385,0.077619,0.131429,25.0
"[25.0, 35.0)",age.in.years,399,0.399,268.0,131.0,0.328321,0.131508,0.007076,0.089385,0.131429,0.131429,35.0
"[35.0, 45.0)",age.in.years,251,0.251,193.0,58.0,0.231076,-0.354949,0.029241,0.089385,0.049048,0.131429,45.0
"[45.0, 55.0)",age.in.years,122,0.122,94.0,28.0,0.229508,-0.363792,0.014898,0.089385,0.008095,0.131429,55.0
"[55.0, inf)",age.in.years,79,0.079,57.0,22.0,0.278481,-0.104711,0.000848,0.089385,0.000000,0.131429,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,missing


当然也支持样本权重

In [23]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index)
bm.varReportSinge().report(X['age.in.years'],y,[25,35,45,55],sample_weight)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 25.0)",age.in.years,941,0.128904,880.0,61.0,0.064825,0.480835,0.037322,0.089385,0.077619,0.131429,25.0
"[25.0, 35.0)",age.in.years,2811,0.385068,2680.0,131.0,0.046603,0.131508,0.007076,0.089385,0.131429,0.131429,35.0
"[35.0, 45.0)",age.in.years,1988,0.272329,1930.0,58.0,0.029175,-0.354949,0.029241,0.089385,0.049048,0.131429,45.0
"[45.0, 55.0)",age.in.years,968,0.132603,940.0,28.0,0.028926,-0.363792,0.014898,0.089385,0.008095,0.131429,55.0
"[55.0, inf)",age.in.years,592,0.081096,570.0,22.0,0.037162,-0.104711,0.000848,0.089385,0.000000,0.131429,inf
special,age.in.years,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,special
missing,age.in.years,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,missing


## varGroupsReport

在实践中,单一的特征分析报告往往仍旧无法满足实际需要,往往希望通过不同的月份\产品\客群查看各个策略的表现,那么就需要分组产生分析报告

varGroupsReport就是用来满足这种需要的

### 基本用法

In [24]:
#模拟月份(9,10,11,12月),模拟客群(3个)

X_all=X.join(y).assign(
    month=np.random.randint(9,13,y.size),
    client_group=pd.Series(np.random.randint(0,3,y.size),index=y.index).map({0:'g1',1:'g2',2:'g3'})
)

In [25]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,row_limit=0,
                          n_jobs=1).fit(X_all)

vtab_g中共五张报表:
+ report_all:各个组的特征报告
+ report_brief:各个组的简化版报告,只保留count,badprob,woe,total_iv,ks_max
+ report_count:各个组的简化版报告,只保留count
+ report_badprob:各个组的简化版报告,只保留badprob
+ report_iv:各个组的简化版报告,只保留total_iv
+ report_ks:各个组的简化版报告,只保留ks_max

以report_brief为例

In [26]:
vtab_g.report_dict['report_brief'][['variable','bin','10']] #只看10月份的报告

variable  \
                                           
                                           
0                           age.in.years   
1                           age.in.years   
2                           age.in.years   
3                           age.in.years   
4                           age.in.years   
..                                   ...   
115  status.of.existing.checking.account   
116                            telephone   
117                            telephone   
118                            telephone   
119                            telephone   

                                          bin    10                       \
                                                 g1                        
                                              count   badprob        woe   
0                                [-inf, 26.0)    12  0.583333   1.402824   
1                                [26.0, 28.0)    14  0.142857  -0.725408   
2                                [28.0, 30.0)     5  0.400000   0.660886   
3                                [30.0, 35.0)    18  0.277778   0.110840   
4                                [35.0, 37.0)     5  0.000000 -20.524415   
..                                        ...   ...       ...        ...   
115                                   missing     0       NaN   0.000000   
116  yes, registered under the customers name    48  0.166667  -0.543086   
117                                      none    34  0.382353   0.586778   
118                                   special     0       NaN   0.000000   
119                                   missing     0       NaN   0.000000   

                                                                             \
                           g2                                            g3   
     total_iv    ks_max count   badprob       woe  total_iv    ks_max count   
0    2.269048  0.251366    19  0.368421  0.276040  0.573315  0.207650    21   
1    2.269048  0.251366    13  0.230769 -0.388936  0.573315  0.207650     9   
2    2.269048  0.251366     7  0.714286  1.731328  0.573315  0.207650     3   
3    2.269048  0.251366     7  0.428571  0.527355  0.573315  0.207650    23   
4    2.269048  0.251366    10  0.100000 -1.382188  0.573315  0.207650     6   
..        ...       ...   ...       ...       ...       ...       ...   ...   
115  0.865084  0.432475     0       NaN  0.000000  0.832895  0.388585     0   
116  0.310470  0.274785    36  0.250000 -0.283575  0.050560  0.109290    43   
117  0.310470  0.274785    52  0.346154  0.179048  0.050560  0.109290    51   
118  0.310470  0.274785     0       NaN  0.000000  0.050560  0.109290     0   
119  0.310470  0.274785     0       NaN  0.000000  0.050560  0.109290     0   

                                              
                                              
      badprob        woe  total_iv    ks_max  
0    0.571429   0.855666  5.028137  0.202941  
1    0.333333  -0.125163  5.028137  0.202941  
2    0.000000 -20.030119  5.028137  0.202941  
3    0.347826  -0.060625  5.028137  0.202941  
4    0.000000 -20.723266  5.028137  0.202941  
..        ...        ...       ...       ...  
115       NaN   0.000000  4.198442  0.536275  
116  0.302326  -0.268264  0.056420  0.117647  
117  0.411765   0.211309  0.056420  0.117647  
118       NaN   0.000000  0.056420  0.117647  
119       NaN   0.000000  0.056420  0.117647  

[120 rows x 17 columns]

In [27]:
vtab_g.report_dict['report_brief'] #全部报告

variable  \
                                           
                                           
0                           age.in.years   
1                           age.in.years   
2                           age.in.years   
3                           age.in.years   
4                           age.in.years   
..                                   ...   
115  status.of.existing.checking.account   
116                            telephone   
117                            telephone   
118                            telephone   
119                            telephone   

                                          bin    10                       \
                                                 g1                        
                                              count   badprob        woe   
0                                [-inf, 26.0)    12  0.583333   1.402824   
1                                [26.0, 28.0)    14  0.142857  -0.725408   
2                                [28.0, 30.0)     5  0.400000   0.660886   
3                                [30.0, 35.0)    18  0.277778   0.110840   
4                                [35.0, 37.0)     5  0.000000 -20.524415   
..                                        ...   ...       ...        ...   
115                                   missing     0       NaN   0.000000   
116  yes, registered under the customers name    48  0.166667  -0.543086   
117                                      none    34  0.382353   0.586778   
118                                   special     0       NaN   0.000000   
119                                   missing     0       NaN   0.000000   

                                                   ...     9            \
                           g2                      ...    g2             
     total_iv    ks_max count   badprob       woe  ... count   badprob   
0    2.269048  0.251366    19  0.368421  0.276040  ...    16  0.562500   
1    2.269048  0.251366    13  0.230769 -0.388936  ...     7  0.142857   
2    2.269048  0.251366     7  0.714286  1.731328  ...     6  0.166667   
3    2.269048  0.251366     7  0.428571  0.527355  ...    18  0.444444   
4    2.269048  0.251366    10  0.100000 -1.382188  ...     4  0.000000   
..        ...       ...   ...       ...       ...  ...   ...       ...   
115  0.865084  0.432475     0       NaN  0.000000  ...     0       NaN   
116  0.310470  0.274785    36  0.250000 -0.283575  ...    32  0.343750   
117  0.310470  0.274785    52  0.346154  0.179048  ...    56  0.321429   
118  0.310470  0.274785     0       NaN  0.000000  ...     0       NaN   
119  0.310470  0.274785     0       NaN  0.000000  ...     0       NaN   

                                                                         \
                                      g3                                  
           woe  total_iv    ks_max count   badprob        woe  total_iv   
0     0.961556  1.829467  0.191701    17  0.352941   0.106060  1.718245   
1    -1.081518  1.829467  0.191701     9  0.111111  -1.367246  1.718245   
2    -0.899196  1.829467  0.191701     7  0.714286   1.628486  1.718245   
3     0.487098  1.829467  0.191701    16  0.375000   0.201370  1.718245   
4   -20.334608  1.829467  0.191701     1  0.000000 -19.055559  1.718245   
..         ...       ...       ...   ...       ...        ...       ...   
115   0.000000  0.824997  0.369959     0       NaN   0.000000  0.877798   
116   0.063614  0.002352  0.023378    31  0.419355   0.386773  0.106647   
117  -0.036973  0.002352  0.023378    48  0.270833  -0.278203  0.106647   
118   0.000000  0.002352  0.023378     0       NaN   0.000000  0.106647   
119   0.000000  0.002352  0.023378     0       NaN   0.000000  0.106647   

               
               
       ks_max  
0    0.107402  
1    0.107402  
2    0.107402  
3    0.107402  
4    0.107402  
..        ...  
115  0.394049  
116  0.160377  
117  0.160377  
118  0.160377  
119  0.160377  

[120 rows x 62 columns]

In [28]:
vtab_g.report_dict['report_ks']

variable        10            \
                                                             g1        g2   
                                                         ks_max    ks_max   
0                                        age.in.years  0.251366  0.207650   
1                                       credit.amount  0.298205  0.107468   
2                                      credit.history  0.286495  0.195507   
3                                   duration.in.month  0.237315  0.193685   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.312256  0.034001   
6   installment.rate.in.percentage.of.disposable.i...  0.051522  0.094111   
7                                                 job  0.086651  0.033394   
8             number.of.existing.credits.at.this.bank  0.018735  0.060109   
9   number.of.people.being.liable.to.provide.maint...  0.028103  0.036430   
10                        other.debtors.or.guarantors  0.065574  0.012143   
11                            other.installment.plans  0.249805  0.099575   
12                            personal.status.and.sex  0.050742  0.065574   
13                           present.employment.since  0.123341  0.142077   
14                            present.residence.since  0.176425  0.228294   
15                                           property  0.227166  0.219794   
16                                            purpose  0.155347  0.204007   
17                          savings.account.and.bonds  0.247463  0.216151   
18                status.of.existing.checking.account  0.432475  0.388585   
19                                          telephone  0.274785  0.109290   

                    11                            12                      \
          g3        g1        g2        g3        g1        g2        g3   
      ks_max    ks_max    ks_max    ks_max    ks_max    ks_max    ks_max   
0   0.202941  0.075901  0.189840  0.153333  0.113290  0.277961  0.248284   
1   0.132353  0.126186  0.156863  0.193333  0.305011  0.317434  0.233410   
2   0.152941  0.109108  0.187611  0.176667  0.324619  0.175987  0.115561   
3   0.081373  0.242884  0.197861  0.156111  0.237473  0.388980  0.240275   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.144118  0.190702  0.128342  0.050000  0.263617  0.013158  0.181922   
6   0.091176  0.211575  0.180481  0.034444  0.198257  0.115954  0.212815   
7   0.051961  0.159393  0.010250  0.099444  0.117647  0.296875  0.097254   
8   0.089216  0.104364  0.110517  0.143333  0.169935  0.129934  0.185355   
9   0.011765  0.165085  0.011141  0.034444  0.119826  0.119243  0.053776   
10  0.100000  0.021822  0.045900  0.043333  0.078431  0.005757  0.026316   
11  0.035294  0.357685  0.035651  0.195000  0.100218  0.013980  0.051487   
12  0.054902  0.239089  0.017380  0.062778  0.093682  0.168586  0.159039   
13  0.253922  0.293169  0.121658  0.207222  0.082789  0.249178  0.117849   
14  0.070588  0.243833  0.063725  0.086111  0.202614  0.079770  0.070938   
15  0.257843  0.163188  0.142602  0.170556  0.176471  0.141447  0.240275   
16  0.176471  0.110057  0.122549  0.226667  0.420479  0.219572  0.221968   
17  0.076471  0.371917  0.131462  0.242778  0.108932  0.285362  0.377574   
18  0.536275  0.301708  0.359626  0.357222  0.361656  0.357730  0.430206   
19  0.117647  0.018027  0.092246  0.148333  0.267974  0.092928  0.056064   

           9                      
          g1        g2        g3  
      ks_max    ks_max    ks_max  
0   0.145511  0.191701  0.107402  
1   0.336429  0.216832  0.140784  
2   0.326109  0.206897  0.174165  
3   0.213622  0.346581  0.333817  
4   0.000000  0.000000  0.000000  
5   0.212590  0.228521  0.024673  
6   0.102167  0.149620  0.179245  
7   0.163055  0.099942  0.156749  
8   0.021672  0.005260  0.030479  
9   0.163055  0.122735  0.053701  
10  0.046440  0.033314  0.000726  
11  0.007224  0.105786  0.060232  

### 排序组特征水平

上述报告中,月份排序有一定问题,那么若希望以指定顺序排序报告则可通过sort_columns参数设定 

In [29]:
sort_columns={
    'month':['9','10','11','12'],
    'client_group':['g3','g2','g1']
} 

In [30]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,row_limit=0,
                          sort_columns=sort_columns,
                          n_jobs=1).fit(X_all)

In [31]:
vtab_g.report_dict['report_ks'] #排序后的数据

variable         9            \
                                                             g3        g2   
                                                         ks_max    ks_max   
0                                        age.in.years  0.107402  0.191701   
1                                       credit.amount  0.140784  0.216832   
2                                      credit.history  0.174165  0.206897   
3                                   duration.in.month  0.333817  0.346581   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.024673  0.228521   
6   installment.rate.in.percentage.of.disposable.i...  0.179245  0.149620   
7                                                 job  0.156749  0.099942   
8             number.of.existing.credits.at.this.bank  0.030479  0.005260   
9   number.of.people.being.liable.to.provide.maint...  0.053701  0.122735   
10                        other.debtors.or.guarantors  0.000726  0.033314   
11                            other.installment.plans  0.060232  0.105786   
12                            personal.status.and.sex  0.074746  0.163647   
13                           present.employment.since  0.138607  0.168907   
14                            present.residence.since  0.045718  0.058445   
15                                           property  0.023948  0.192285   
16                                            purpose  0.107402  0.250731   
17                          savings.account.and.bonds  0.144412  0.267680   
18                status.of.existing.checking.account  0.394049  0.369959   
19                                          telephone  0.160377  0.023378   

                    10                            11                      \
          g1        g3        g2        g1        g3        g2        g1   
      ks_max    ks_max    ks_max    ks_max    ks_max    ks_max    ks_max   
0   0.145511  0.202941  0.207650  0.251366  0.153333  0.189840  0.075901   
1   0.336429  0.132353  0.107468  0.298205  0.193333  0.156863  0.126186   
2   0.326109  0.152941  0.195507  0.286495  0.176667  0.187611  0.109108   
3   0.213622  0.081373  0.193685  0.237315  0.156111  0.197861  0.242884   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.212590  0.144118  0.034001  0.312256  0.050000  0.128342  0.190702   
6   0.102167  0.091176  0.094111  0.051522  0.034444  0.180481  0.211575   
7   0.163055  0.051961  0.033394  0.086651  0.099444  0.010250  0.159393   
8   0.021672  0.089216  0.060109  0.018735  0.143333  0.110517  0.104364   
9   0.163055  0.011765  0.036430  0.028103  0.034444  0.011141  0.165085   
10  0.046440  0.100000  0.012143  0.065574  0.043333  0.045900  0.021822   
11  0.007224  0.035294  0.099575  0.249805  0.195000  0.035651  0.357685   
12  0.109391  0.054902  0.065574  0.050742  0.062778  0.017380  0.239089   
13  0.062951  0.253922  0.142077  0.123341  0.207222  0.121658  0.293169   
14  0.036120  0.070588  0.228294  0.176425  0.086111  0.063725  0.243833   
15  0.125903  0.257843  0.219794  0.227166  0.170556  0.142602  0.163188   
16  0.383901  0.176471  0.204007  0.155347  0.226667  0.122549  0.110057   
17  0.083591  0.076471  0.216151  0.247463  0.242778  0.131462  0.371917   
18  0.291022  0.536275  0.388585  0.432475  0.357222  0.359626  0.301708   
19  0.035088  0.117647  0.109290  0.274785  0.148333  0.092246  0.018027   

          12                      
          g3        g2        g1  
      ks_max    ks_max    ks_max  
0   0.248284  0.277961  0.113290  
1   0.233410  0.317434  0.305011  
2   0.115561  0.175987  0.324619  
3   0.240275  0.388980  0.237473  
4   0.000000  0.000000  0.000000  
5   0.181922  0.013158  0.263617  
6   0.212815  0.115954  0.198257  
7   0.097254  0.296875  0.117647  
8   0.185355  0.129934  0.169935  
9   0.053776  0.119243  0.119826  
10  0.026316  0.005757  0.078431  
11  0.051487  0.013980  0.100218  

+ 若希望client_group在第一行,month在第二行,那么只需要把columns=['month','client_group']换为columns=['client_group','month']

### 使用row_limit

有时组分得过细会导致组内样本量不足,指标可能不具备代表性,因此可通过row_limit进行限定,若组数量小于row_limit时,该组就不会统计其任何指标

In [32]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,
                          row_limit=80,
                          sort_columns=sort_columns,
                          n_jobs=1).fit(X_all)

group ('11', 'g1') has rows less than 80,output will return None
group ('12', 'g1') has rows less than 80,output will return None
group ('12', 'g3') has rows less than 80,output will return None
group ('9', 'g1') has rows less than 80,output will return None
group ('9', 'g3') has rows less than 80,output will return None


可以看到警告信息中相应剔除的组的信息

In [33]:
vtab_g.report_dict['report_ks']

variable         9        10  \
                                                             g2        g3   
                                                         ks_max    ks_max   
0                                        age.in.years  0.191701  0.202941   
1                                       credit.amount  0.216832  0.132353   
2                                      credit.history  0.206897  0.152941   
3                                   duration.in.month  0.346581  0.081373   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.228521  0.144118   
6   installment.rate.in.percentage.of.disposable.i...  0.149620  0.091176   
7                                                 job  0.099942  0.051961   
8             number.of.existing.credits.at.this.bank  0.005260  0.089216   
9   number.of.people.being.liable.to.provide.maint...  0.122735  0.011765   
10                        other.debtors.or.guarantors  0.033314  0.100000   
11                            other.installment.plans  0.105786  0.035294   
12                            personal.status.and.sex  0.163647  0.054902   
13                           present.employment.since  0.168907  0.253922   
14                            present.residence.since  0.058445  0.070588   
15                                           property  0.192285  0.257843   
16                                            purpose  0.250731  0.176471   
17                          savings.account.and.bonds  0.267680  0.076471   
18                status.of.existing.checking.account  0.369959  0.536275   
19                                          telephone  0.023378  0.117647   

                              11                  12  
          g2        g1        g3        g2        g2  
      ks_max    ks_max    ks_max    ks_max    ks_max  
0   0.207650  0.251366  0.153333  0.189840  0.277961  
1   0.107468  0.298205  0.193333  0.156863  0.317434  
2   0.195507  0.286495  0.176667  0.187611  0.175987  
3   0.193685  0.237315  0.156111  0.197861  0.388980  
4   0.000000  0.000000  0.000000  0.000000  0.000000  
5   0.034001  0.312256  0.050000  0.128342  0.013158  
6   0.094111  0.051522  0.034444  0.180481  0.115954  
7   0.033394  0.086651  0.099444  0.010250  0.296875  
8   0.060109  0.018735  0.143333  0.110517  0.129934  
9   0.036430  0.028103  0.034444  0.011141  0.119243  
10  0.012143  0.065574  0.043333  0.045900  0.005757  
11  0.099575  0.249805  0.195000  0.035651  0.013980  
12  0.065574  0.050742  0.062778  0.017380  0.168586  
13  0.142077  0.123341  0.207222  0.121658  0.249178  
14  0.228294  0.176425  0.086111  0.063725  0.079770  
15  0.219794  0.227166  0.170556  0.142602  0.141447  
16  0.204007  0.155347  0.226667  0.122549  0.219572  
17  0.216151  0.247463  0.242778  0.131462  0.285362  
18  0.388585  0.432475  0.357222  0.359626  0.357730  
19  0.109290  0.274785  0.148333  0.092246  0.092928

### 产生psi报告

varGroupsReport可以产生psi报告以比较各个组中各个变量的分布变动情况

In [34]:
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],target=y.name,
                    row_limit=0,output_psi=True,n_jobs=1).fit(X_all)

In [35]:
vtabs_g.report_dict['report_psi'].head(21)

,variable,bin,g1,g2,g3
,,,count_distr,count_distr,count_distr
0,age.in.years,"[-inf, 26.0)",0.171521,0.180556,0.217523
1,age.in.years,"[26.0, 28.0)",0.097087,0.113889,0.090634
2,age.in.years,"[28.0, 30.0)",0.077670,0.094444,0.066465
3,age.in.years,"[30.0, 35.0)",0.210356,0.130556,0.196375
4,age.in.years,"[35.0, 37.0)",0.084142,0.108333,0.042296
5,age.in.years,"[37.0, 48.0)",0.194175,0.216667,0.253776
6,age.in.years,"[48.0, 53.0)",0.055016,0.061111,0.048338
7,age.in.years,"[53.0, inf)",0.110032,0.094444,0.084592
8,age.in.years,missing,0.000000,0.000000,0.000000


每一个变量的分组最后一行都会有psi用于显示各个组与基准分布比较而产生的psi

默认情况下psi的基准分布为全量数据分布,这里也可以指定某一组的分布为基准分布

+ 参数psi_base用于指定分布基准
+ 参数psi_base会在模块内部通过X.query(psi_base)传递,其语法为pd.DataFrame.query()的语法

In [36]:
X_g_gen=X_all.groupby('client_group')

In [37]:
#选择client_group=="g1"为psi基准分布
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],target=y.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=1).fit(X_all)

In [38]:
vtabs_g.report_dict['report_psi'].head(20) #可以看到g1的psi为0

,variable,bin,g1,g2,g3
,,,count_distr,count_distr,count_distr
0,age.in.years,"[-inf, 26.0)",0.171521,0.180556,0.217523
1,age.in.years,"[26.0, 28.0)",0.097087,0.113889,0.090634
2,age.in.years,"[28.0, 30.0)",0.077670,0.094444,0.066465
3,age.in.years,"[30.0, 35.0)",0.210356,0.130556,0.196375
4,age.in.years,"[35.0, 37.0)",0.084142,0.108333,0.042296
5,age.in.years,"[37.0, 48.0)",0.194175,0.216667,0.253776
6,age.in.years,"[48.0, 53.0)",0.055016,0.061111,0.048338
7,age.in.years,"[53.0, inf)",0.110032,0.094444,0.084592
8,age.in.years,missing,0.000000,0.000000,0.000000


### 并行

varGroupsReport使用joblib对各组的报告生成进行了并行优化,

+ 当基础的数据量非常大列较多组的数量较多时可通过设定n_jobs=任务数进行并行以提高运行速度
+ 一般情况下建议n_jobs=1

In [39]:
#模拟较大数据量,3w行1000个特征,一个组特征(5个水平)
X_big=pd.DataFrame(np.random.rand(30000,1000),columns=['f'+str(i) for i in range(1000)])
y_big=pd.Series(np.random.randint(0,2,30000),name='target')

breaks_list_big={col:[0.2,0.4,0.6,0.8] for col in X_big.columns}

X_big_all=X_big.join(y_big).assign(
    client_group=pd.Series(np.random.randint(0,5,y_big.size),index=y_big.index).map({0:'g1',
                                                                                     1:'g2',
                                                                                     2:'g3',
                                                                                     3:'g4',
                                                                                     4:'g5'
                                                                                    })
)

In [40]:
X_big_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1002 entries, f0 to client_group
dtypes: float64(1000), int64(1), object(1)
memory usage: 229.3+ MB


In [41]:
%%time
vtabs_g_p=bm.varGroupsReport(breaks_list_big,columns=['client_group'],target=y_big.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=-1).fit(X_big_all)

CPU times: user 1min 11s, sys: 1min 16s, total: 2min 27s
Wall time: 2min 55s


In [42]:
%%time
vtabs_g=bm.varGroupsReport(breaks_list_big,columns=['client_group'],target=y_big.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=1).fit(X_big_all)

CPU times: user 2min 26s, sys: 1min 13s, total: 3min 40s
Wall time: 3min 42s


### 样本权重

varGroupsReport提供了参数sample_weight,以在报告中加入样本权重,使用方法与varReport一致

In [43]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index) #假定数据经过抽样后好样本权重为10,坏样本权重为1

In [44]:
vtab_g_ws=bm.varGroupsReport(bin_tree.breaks_list,columns=['split'],target=y.name,
                       sample_weight=sample_weight,
                       row_limit=0,n_jobs=1).fit(X_all.assign(split=1))

In [45]:
vtab_g_ws.report_dict['report_all'].head(10)

variable           bin     1                                      \
                              count count_distr    good   bad   badprob   
0  age.in.years  [-inf, 26.0)  1180    0.161644  1100.0  80.0  0.067797   
1  age.in.years  [26.0, 28.0)   767    0.105068   740.0  27.0  0.035202   
2  age.in.years  [28.0, 30.0)   530    0.072603   500.0  30.0  0.056604   
3  age.in.years  [30.0, 35.0)  1275    0.174658  1220.0  55.0  0.043137   
4  age.in.years  [35.0, 37.0)   682    0.093425   670.0  12.0  0.017595   
5  age.in.years  [37.0, 48.0)  1698    0.232603  1640.0  58.0  0.034158   
6  age.in.years  [48.0, 53.0)   469    0.064247   460.0   9.0  0.019190   
7  age.in.years   [53.0, inf)   699    0.095753   670.0  29.0  0.041488   
8  age.in.years       special     0    0.000000     0.0   0.0       NaN   
9  age.in.years       missing     0    0.000000     0.0   0.0       NaN   

                                                              
        woe    bin_iv  total_iv        ks    ks_max   breaks  
0  0.528844  0.057921  0.155013  0.109524  0.131429     26.0  
1 -0.160930  0.002529  0.155013  0.093810  0.131429     28.0  
2  0.336472  0.009613  0.155013  0.122381  0.131429     30.0  
3  0.050610  0.000458  0.155013  0.131429  0.131429     35.0  
4 -0.872488  0.048610  0.155013  0.075714  0.131429     37.0  
5 -0.192126  0.007868  0.155013  0.034762  0.131429     48.0  
6 -0.784119  0.028004  0.155013  0.000952  0.131429     53.0  
7  0.009901  0.000009  0.155013  0.000000  0.131429      inf  
8  0.000000  0.000000  0.155013  0.000000  0.131429  special  
9  0.000000  0.000000  0.155013  0.000000  0.131429  missing

### 导出为excel

varGroupsReport支持对组报告导出为excel,名称为var_report.xlsx

+ 参数out_path为报告输出路径,若无此路径则模块会创建该路径
+ 参数tab_suffix代表报告名称后缀,例如tab_suffix=‘_group’时,输出报告名称为var_report_group.xlsx
+ report_dict中所有的报告都会被导出,并写为不同的sheet

In [46]:
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],
                           target=y.name,row_limit=0,
                           output_psi=True,n_jobs=1,
                           out_path='report/',
                           tab_suffix='_client'
                          ).fit(X_all)

to_excel done
